1. Решить проблему с GCV
2. Почему подтянулось на одно значение меньше
3. Пофиксить нули в изначальных данных? Что делать, если GCV тоже будет нулевое?
4. Добавить данные через request
5. Залить на github

In [1]:
import pandas as pd

In [2]:
WH_flow_df = pd.read_csv('C:/Users/user/Documents/IDE/data/NG/Wilhelmshaven_flow_2023_05_12_160150.csv')
WH_GCV_df = pd.read_csv('C:/Users/user/Documents/IDE/data/NG/Wilhelmshaven_GCV__2023_05_12_160436.csv')
BEG_flow_df = pd.read_csv('C:/Users/user/Documents/IDE/data/NG/Baltic_Energy_Gate_flow_2023_05_12_160908.csv')
BEG_GCV_df = pd.read_csv('C:/Users/user/Documents/IDE/data/NG/Baltic_Energy_Gate_GCV__2023_05_12_160946.csv')
BBH_flow_df = pd.read_csv('C:/Users/user/Documents/IDE/data/NG/BRUNSBUETTEL_HAFEN_flow_2023_05_12_162127.csv')
BBH_GCV_df = pd.read_csv('C:/Users/user/Documents/IDE/data/NG/BRUNSBUETTEL_HAFEN_GCV_2023_05_12_161110.csv')


In [3]:
# I will use "periodFrom" as date series "index".
# Converting columns to datetime format.
WH_flow_df['periodFrom'] = pd.to_datetime(WH_flow_df['periodFrom'],dayfirst=False)
WH_GCV_df['periodFrom'] = pd.to_datetime(WH_GCV_df['periodFrom'],dayfirst=False)
BEG_flow_df['periodFrom'] = pd.to_datetime(BEG_flow_df['periodFrom'],dayfirst=False)
BEG_GCV_df['periodFrom'] = pd.to_datetime(BEG_GCV_df['periodFrom'],dayfirst=False)
BBH_flow_df['periodFrom'] = pd.to_datetime(BBH_flow_df['periodFrom'],dayfirst=False)
BBH_GCV_df['periodFrom'] = pd.to_datetime(BBH_GCV_df['periodFrom'],dayfirst=False)

In [4]:
display(BBH_flow_df.info())
display(BBH_flow_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   id                             100 non-null    object        
 1   dataSet                        100 non-null    int64         
 2   indicator                      100 non-null    object        
 3   periodType                     100 non-null    object        
 4   periodFrom                     100 non-null    datetime64[ns]
 5   periodTo                       100 non-null    object        
 6   operatorKey                    100 non-null    object        
 7   tsoEicCode                     100 non-null    object        
 8   operatorLabel                  100 non-null    object        
 9   pointKey                       100 non-null    object        
 10  pointLabel                     100 non-null    object        
 11  tsoItemIdentifier   

None

,id,dataSet,indicator,periodType,periodFrom,periodTo,operatorKey,tsoEicCode,operatorLabel,pointKey,...,isNA,originalPeriodFrom,isCmpRelevant,bookingPlatformKey,bookingPlatformLabel,bookingPlatformURL,interruptionCalculationRemark,pointType,idPointType,isArchived
0,1Physical Flowday2023-02-012023-02-02DE-TSO-00...,1,Physical Flow,day,2023-02-01 06:00:00,2023-02-02 06:00,DE-TSO-0005,21X-DE-D-A0A0A-K,Gasunie Deutschland,LNG-00060,...,NaN,NaN,NaN,PRISMA,NaN,https://platform.prisma-capacity.eu/,NaN,LNG Entry point,2,NaN
1,1Physical Flowday2023-02-022023-02-03DE-TSO-00...,1,Physical Flow,day,2023-02-02 06:00:00,2023-02-03 06:00,DE-TSO-0005,21X-DE-D-A0A0A-K,Gasunie Deutschland,LNG-00060,...,NaN,NaN,NaN,PRISMA,NaN,https://platform.prisma-capacity.eu/,NaN,LNG Entry point,2,NaN
2,1Physical Flowday2023-02-032023-02-04DE-TSO-00...,1,Physical Flow,day,2023-02-03 06:00:00,2023-02-04 06:00,DE-TSO-0005,21X-DE-D-A0A0A-K,Gasunie Deutschland,LNG-00060,...,NaN,NaN,NaN,PRISMA,NaN,https://platform.prisma-capacity.eu/,NaN,LNG Entry point,2,NaN
3,1Physical Flowday2023-02-042023-02-05DE-TSO-00...,1,Physical Flow,day,2023-02-04 06:00:00,2023-02-05 06:00,DE-TSO-0005,21X-DE-D-A0A0A-K,Gasunie Deutschland,LNG-00060,...,NaN,NaN,NaN,PRISMA,NaN,https://platform.prisma-capacity.eu/,NaN,LNG Entry point,2,NaN
4,1Physical Flowday2023-02-052023-02-06DE-TSO-00...,1,Physical Flow,day,2023-02-05 06:00:00,2023-02-06 06:00,DE-TSO-0005,21X-DE-D-A0A0A-K,Gasunie Deutschland,LNG-00060,...,NaN,NaN,NaN,PRISMA,NaN,https://platform.prisma-capacity.eu/,NaN,LNG Entry point,2,NaN


### Solving missing GCV data entries.

During the analysis I encountered this problem that for Baltic Energy Gate Terminal for some days with gas flow data there is now GCV data.  
I looked into it and there is no GCV data for the starting period of Terminal working.  
So, maybe there was some technical problems with data collection.  
Anyway, this problem could happen for any date.  
So I'm going resolve it this way:  
I will merge (by "left join") GCV data with gas flow data for each respective Terminal (keeping in mind that gas flow and GCV data should be downloaded for the same period).    
And then, if the is some missing GCV data I would fill it with the average GCV for the period.  
This is probably not the best option, but it will do for now.  
Maybe I will get back to it later.

In [5]:
# Creating DataFrames for each Terminal

BEG_flow_df = BEG_flow_df[['periodFrom','value']]
BEG_flow_df.rename(columns={"value": "Baltic_Energy_Gate_flow, kWh/d"},inplace=True)

BEG_GCV_df = BEG_GCV_df[['periodFrom','value']]
BEG_GCV_df.rename(columns={"value": "Baltic_Energy_Gate_GCV, kWh/Nm3"},inplace=True)

BEG_df = BEG_flow_df.merge(right=BEG_GCV_df, how='left',left_on='periodFrom',right_on='periodFrom')

In [6]:
WH_flow_df = WH_flow_df[['periodFrom','value']]
WH_flow_df.rename(columns={"value": "Wilhelmshaven_flow, kWh/d"},inplace=True)

WH_GCV_df = WH_GCV_df[['periodFrom','value']]
WH_GCV_df.rename(columns={"value": "Wilhelmshaven_GCV, kWh/Nm3"},inplace=True)

WH_df = WH_flow_df.merge(right=WH_GCV_df, how='left',left_on='periodFrom',right_on='periodFrom')

In [7]:
BBH_flow_df = BBH_flow_df[['periodFrom','value']]
BBH_flow_df.rename(columns={"value": "Brunsbuettel_Hafen_flow, kWh/d"},inplace=True)

BBH_GCV_df = BBH_GCV_df[['periodFrom','value']]
BBH_GCV_df.rename(columns={"value": "Brunsbuettel_Hafen_GCV, kWh/Nm3"},inplace=True)

BBH_df = BBH_flow_df.merge(right=BBH_GCV_df, how='left',left_on='periodFrom',right_on='periodFrom')

In [8]:
# Calculating average GCV for each Terminal
mean_GCV_WH = WH_df['Wilhelmshaven_GCV, kWh/Nm3'].mean(skipna=True)
mean_GCV_BBH = BBH_df['Brunsbuettel_Hafen_GCV, kWh/Nm3'].mean(skipna=True)
mean_GCV_BEG = BEG_df['Baltic_Energy_Gate_GCV, kWh/Nm3'].mean(skipna=True)

In [10]:
# Defining function GCV_fill that will fill missing GCV data
def GCV_fill(entry, Terminal):
    if pd.isna(entry) == True:
        if Terminal == 'WH':
            return mean_GCV_WH
        if Terminal == 'BBH':
            return mean_GCV_BBH
        if Terminal == 'BEG':
            return mean_GCV_BEG
    else:
        return entry

In [11]:
WH_df['Wilhelmshaven_GCV, kWh/Nm3'] = WH_df['Wilhelmshaven_GCV, kWh/Nm3'].apply(GCV_fill, Terminal='WH')
BBH_df['Brunsbuettel_Hafen_GCV, kWh/Nm3'] = BBH_df['Brunsbuettel_Hafen_GCV, kWh/Nm3'].apply(GCV_fill, Terminal='BBH') 
BEG_df['Baltic_Energy_Gate_GCV, kWh/Nm3'] = BEG_df['Baltic_Energy_Gate_GCV, kWh/Nm3'].apply(GCV_fill, Terminal='BEG')  

### Creating DataFrame for the plot

Now I want to create a dataframe where we'll have a date range as "index".  
Then I'll add flows and GCV data by merging ("left join") data to this "index".  
The point is that downloaded dataframeS with terminal's data have different amount of entries,  
as Terminals have gas flows on different dates.  
So this "index" will serve as base for this dataframe.

In [20]:
# Creating date range.
merged_df = pd.DataFrame(pd.date_range(start='2022-09-01 06:00:00', end='2023-05-10 06:00:00', freq='D'), columns=['date'])

In [ ]:
# Merging dataframes.
merged_df = merged_df.merge(right=WH_flow_df, how='left',left_on='date',right_on='periodFrom')
merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
merged_df.rename(columns={"value": "Wilhelmshaven_flow, kWh/d"},inplace=True)

merged_df = merged_df.merge(right=WH_GCV_df, how='left',left_on='date',right_on='periodFrom')
merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
merged_df.rename(columns={"value": "Wilhelmshaven_GCV, kWh/Nm3"},inplace=True)

merged_df = merged_df.merge(right=BEG_flow_df, how='left',left_on='date',right_on='periodFrom')
merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
merged_df.rename(columns={"value": "Baltic_Energy_Gate_flow, kWh/d"},inplace=True)

merged_df = merged_df.merge(right=BEG_GCV_df, how='left',left_on='date',right_on='periodFrom')
merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
merged_df.rename(columns={"value": "Baltic_Energy_Gate_GCV, kWh/Nm3"},inplace=True)

merged_df = merged_df.merge(right=BBH_flow_df, how='left',left_on='date',right_on='periodFrom')
merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
merged_df.rename(columns={"value": "Brunsbuettel_Hafen_flow, kWh/d"},inplace=True)

merged_df = merged_df.merge(right=BBH_GCV_df, how='left',left_on='date',right_on='periodFrom')
merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
merged_df.rename(columns={"value": "Brunsbuettel_Hafen_GCV, kWh/Nm3"},inplace=True)


In [ ]:
# Converting kWh/day to
merged_df['Wilhelmshaven_flow, kWh/d'] = merged_df['Wilhelmshaven_flow, kWh/d']/1000/1000
merged_df['Baltic_Energy_Gate_flow, kWh/d'] = merged_df['Baltic_Energy_Gate_flow, kWh/d']/1000/1000
merged_df['Brunsbuettel_Hafen_flow, kWh/d'] = merged_df['Brunsbuettel_Hafen_flow, kWh/d']/1000/1000

In [ ]:
# Renaming columns accordingly
merged_df.rename(columns={"Wilhelmshaven_flow, kWh/d": "Wilhelmshaven_flow, GWh/d"},inplace=True)
merged_df.rename(columns={"Baltic_Energy_Gate_flow, kWh/d": "Baltic_Energy_Gate_flow, GWh/d"},inplace=True)
merged_df.rename(columns={"Brunsbuettel_Hafen_flow, kWh/d": "Brunsbuettel_Hafen_flow, GWh/d"},inplace=True)

In [ ]:
display(merged_df.head())
display(merged_df.info())
display(merged_df[merged_df['date'] == '2023-04-01 06:00:00'])

Now I want to create visualization for these gas flows using matplotlib library.  
I want to create two stackplots for flow in GWh/day and in 1000 m3/day.

In [ ]:
display(merged_df[merged_df['Baltic_Energy_Gate_GCV, kWh/Nm3'].isnull() == True])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#fig, ax = plt.subplots()
fig = plt.figure(figsize=(8, 4))
ax = fig.add_axes([0, 0, 1, 1])
ax.stackplot(merged_df['date'], merged_df['Wilhelmshaven_flow, GWh/d'], merged_df['Brunsbuettel_Hafen_flow, GWh/d'], merged_df['Baltic_Energy_Gate_flow, GWh/d'],
             labels=['Wilhelmshaven', 'Brunsbuettel','Baltic_Energy_Gate'], alpha=0.8,
             #colors = ['mediumblue', 'darkred', 'lawngreen']
             )
ax.legend(loc='upper left')
ax.set_title('Gas Flows at German LNG Terminals by day, GWh/day')
#ax.set_xlabel('Day')
ax.set_ylabel('GWh/day')

plt.show()